In [1]:
import tensorflow as tf
import numpy as np
import itertools
import h5py as h5
import matplotlib.pyplot as plt
from matplotlib import pyplot

In [2]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score

In [3]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras import initializers, optimizers
from keras.layers import Input, BatchNormalization
from keras.layers import  Dense, Flatten, Activation, Dropout, Embedding
from keras.layers import LSTM, TimeDistributed, Permute,Reshape, Lambda, RepeatVector, merge, Input,Multiply
from keras.utils import np_utils
from keras.layers.wrappers import  Bidirectional
from keras.layers import *
from keras import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2 
from keras.models import load_model
import keras

Using TensorFlow backend.


In [4]:
from os import listdir
from os.path import isfile, join
import glob
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [5]:
mypath = '/home/chenming/ncrna/ncRDeep2/Data_Processing/EIIP_data/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [6]:
my_files8 = glob.glob("/home/chenming/ncrna/ncRDeep2/Data_Processing/EIIP_data/T*.h5")
my_files8.sort()
my_files8[0:2]

['/home/chenming/ncrna/ncRDeep2/Data_Processing/EIIP_data/Test_0.h5',
 '/home/chenming/ncrna/ncRDeep2/Data_Processing/EIIP_data/Test_1.h5']

In [7]:
def get_file8(fold_no): #get train and test data from file by fold number
    hf_Train = h5.File(my_files8[10+fold_no],'r')
    hf_Test = h5.File(my_files8[fold_no],'r')
    X_train = hf_Train['Train_Data'] # Get train set
    X_train = np.array(X_train)
    Y_train = hf_Train['Label']      # Get train label
    Y_train = np.array(Y_train)
    X_test = hf_Test['Test_Data']     # Get test set
    X_test = np.array(X_test)
    Y_test = hf_Test['Label']       # Get test label
    Y_test = np.array(Y_test)
    Y_train = np_utils.to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = np_utils.to_categorical(Y_test, 13)    #  Process the label of test
    return X_train, Y_train, X_test, Y_test

In [8]:
my_files8[20]
hf_val = h5.File(my_files8[20],'r')
X_val = hf_val['Train_Data']     # Get test set
X_val = np.array(X_val)
Y_val = hf_val['Label']       # Get test label
Y_val = np.array(Y_val)
Y_val = np_utils.to_categorical(Y_val, 13)    #  Process the label of test
X_val[:,:,0:4].shape

(2600, 750, 4)

In [9]:
X_val[0,0,:]

array([0.    , 1.    , 0.    , 0.    , 0.0806])

In [10]:
class_names = ['5S_rRNA', '5.8S_rRNA', 'tRNA', 'ribozymes', 'CD-box', 'miRNA', 'Intron_gpI', 'Intron_gpII', 'HACA-box', 'riboswitch', 'IRES', 'leader', 'scaRNA']
class_names

['5S_rRNA',
 '5.8S_rRNA',
 'tRNA',
 'ribozymes',
 'CD-box',
 'miRNA',
 'Intron_gpI',
 'Intron_gpII',
 'HACA-box',
 'riboswitch',
 'IRES',
 'leader',
 'scaRNA']

In [11]:
def plot_confusion_matrix(cm, classes,
                            normalize=False,
                            title='Confusion matrix',
                            cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    FONT_SIZE = 10
    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix\n============================")
    else:
        #cm = np.asfarray(cm,float64)
        print('Confusion matrix, without normalization\n============================')
    #print(cm)
    plt.figure(figsize=(5*2, 4*2))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=FONT_SIZE)
    plt.yticks(tick_marks, classes, fontsize=FONT_SIZE)
    fmt = '.3f' if normalize else '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                    horizontalalignment="center",
                    fontsize=FONT_SIZE,
                    color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label', fontsize=FONT_SIZE)
    plt.xlabel('Predicted label', fontsize=FONT_SIZE)
    plt.savefig('Conf_mat_avg.pdf')
    plt.show()

In [12]:
def Conv1Dme(f, k, x):
    x1=Conv1D(filters=f,kernel_size=k,strides=1,padding="same",kernel_initializer=initializers.random_uniform()) (x)
    x1 = BatchNormalization()(x1)
    x1=Dropout(0.2)(x1)
    x1=Activation('relu')(x1)
    #x1=MaxPooling1D(pool_size=2, strides=2)(x1)
    return x1

In [13]:
def dense_block_1(xin, f, k, p):
    f1 = f
    k1 = k
    p1 = p

    x1 = Conv1Dme(f1, k1, xin)
    x11 = Conv1Dme(f1, k1, x1)
    x11 = Concatenate(axis=-1)([x1,x11])
    x11 = Conv1Dme(f1, k1, x11)
    x11 = Concatenate(axis=-1)([x1,x11])
    x11 = Conv1Dme(f1, k1, x11)
    x1=MaxPooling1D(pool_size=p1, strides=p1)(x11)
    
    return x1

In [14]:
def fc_block_1(xin, n):
    xf=Dense(n,)(xin)
    xf = BatchNormalization()(xf)
    xf=Dropout(0.2)(xf)
    xf=Activation('relu')(xf)
    return xf

In [15]:
def model_dense5(): # Model
    inputs1 = Input(shape=(750, 4))
    inputs2 = Input(shape=(750, 1))
    
    ###############################################################
    #          1st dense block
    
    x1 = Conv1Dme(64, 5, inputs1)
    x1 = MaxPooling1D(pool_size=2, strides=2)(x1)
    
    x1 = dense_block_1(x1, 128, 5, 4)
    x1 = dense_block_1(x1, 128, 5, 4)
    x1 = Conv1Dme(64, 5, x1)
    x1 = MaxPooling1D(pool_size=2, strides=2)(x1)
    
    
    ###############################################################
    #          2nd dense block
    
    x2 = Conv1Dme(64, 5, inputs2)
    x2 = MaxPooling1D(pool_size=2, strides=2)(x2)
    
    x2 = dense_block_1(x2, 128, 5, 4)
    x2 = dense_block_1(x2, 128, 5, 4)
    x2 = Conv1Dme(64, 5, x2)
    x2 = MaxPooling1D(pool_size=2, strides=2)(x2)
    
    xf=keras.layers.concatenate([x1,x2],axis=-1)
    
    #xf = dense_block_1(xf, 256, 5, 2)
    xf = Conv1Dme(64, 3, xf)
    xf = MaxPooling1D(pool_size=2, strides=2)(xf)
    xf = Flatten()(xf)

    xf=fc_block_1(xf, 256)
    xf=fc_block_1(xf, 64)
    
    xf=Dense(13, activation='softmax',  )(xf)

    model = Model(inputs=[inputs1,inputs2], outputs=xf)

    model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(lr=0.0005),metrics=['accuracy'])
    
    return model

In [18]:
EIIP_train = X_train[:,:,4].reshape((5688,750,1))
EIIP_test = X_test[:,:,4].reshape((632,750,1))

NameError: name 'X_train' is not defined

In [17]:
EIIP_val = X_val[:,:,4].reshape((2600,750,1))

In [33]:
auc_mat_750c1 = []
history_750c1 = []
conf_mat_750c1 = []
history_750c1 = {}
class_report_750c1 = {}
for i in range(10):
    X_train, Y_train, X_test, Y_test = get_file8(i)
    EIIP_train = X_train[:,:,4].reshape((5688,750,1))
    EIIP_test = X_test[:,:,4].reshape((632,750,1))
    model = model_dense5()
    #es = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
    #history = model.fit([X_train[:,:,0:4],EIIP_train], Y_train, validation_data=([X_test[:,:,0:4],EIIP_test], Y_test), epochs=200, verbose=0, callbacks=[es])
    model.load_weights("Checkpoints/Dense_eiip_revise2_%i_fold.h5" %i)
    y = model.predict([X_val[:,:,0:4],EIIP_val])
    y_test_non_category = [ np.argmax(t) for t in Y_val ]
    y_predict_non_category = [ np.argmax(t) for t in y ]
    auc = accuracy_score(y_test_non_category, y_predict_non_category)
    precision,recall,fscore,support=score(y_test_non_category, y_predict_non_category,average='macro')
    mcc = matthews_corrcoef(y_test_non_category, y_predict_non_category)
    conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)
    classification_reports = classification_report(y_test_non_category, y_predict_non_category)
    print(auc,recall,precision,fscore,mcc)
    auc_mat_750c1.append([[auc],[recall],[precision],[fscore],[mcc]])
    #history_5.append([history.history])
    #history_750c1['fold%i'%i]=history.history
    conf_mat_750c1.append([conf_mat])
    class_report_750c1['fold%i'%i]=classification_reports

(0.9461538461538461, 0.9461538461538461, 0.9495917754211582, 0.9464422365207111, 0.9419041349353514)
(0.9403846153846154, 0.9403846153846155, 0.9480493874154676, 0.9418113082968679, 0.9359158048751121)
(0.9426923076923077, 0.9426923076923076, 0.948449838722852, 0.943931594308331, 0.9382589530400763)
(0.9415384615384615, 0.9415384615384614, 0.9480343257188272, 0.943138923725335, 0.9370439628868986)
(0.943076923076923, 0.9430769230769231, 0.9498363442150818, 0.944465684182205, 0.9387456294863125)
(0.9465384615384616, 0.9465384615384613, 0.9535070058287514, 0.9479608888282491, 0.9425240361424327)
(0.9357692307692308, 0.9357692307692308, 0.9417264839760099, 0.9365943568080822, 0.9308162277219465)
(0.9384615384615385, 0.9384615384615386, 0.9433258620943967, 0.9391686418556773, 0.9336368146218113)
(0.9346153846153846, 0.9346153846153845, 0.9454561311142685, 0.9365793293401944, 0.9299271632852552)
(0.9476923076923077, 0.9476923076923076, 0.9504997781883687, 0.9483769317306456, 0.9434648832815

In [34]:
auc_mat_750c1 = np.array(auc_mat_750c1)
for i in range(5):    
    print(np.average(auc_mat_750c1[:,i]))

0.9416923076923076
0.9416923076923075
0.9478476932695182
0.9428469895596299
0.9372237610276715
